In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AmazonGoogleBenchmark

benchmark = AmazonGoogleBenchmark(data_dir_path="../data/")
benchmark

19:10:20 INFO:Extracting Amazon-Google...
19:10:20 INFO:Reading Amazon-Google row_dict...
19:10:20 INFO:Reading Amazon-Google train.csv...
19:10:20 INFO:Reading Amazon-Google valid.csv...
19:10:20 INFO:Reading Amazon-Google test.csv...


<AmazonGoogleBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/amazon_google_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'manufacturer', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

19:10:20 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
19:10:23 INFO:For attr=manufacturer, computing actual max_str_len
19:10:23 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
19:10:23 INFO:For attr=manufacturer, using actual_max_str_len=16
19:10:23 INFO:For attr=price, computing actual max_str_len
19:10:23 INFO:actual_max_str_len=11 must be pair to enable NN pooling. Updating to 12
19:10:23 INFO:For attr=price, using actual_max_str_len=12


{'title': NumericalizeInfo(field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=None, vocab=<torchtext.vocab.Vocab object at 0x7f0bf8136dc0>, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'manufacturer': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'

In [11]:
datamodule = benchmark.build_pairwise_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = 'f1-amzn-googl'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

19:10:23 INFO:GPU available: True, used: True
19:10:23 INFO:TPU available: None, using: 0 TPU cores
19:10:23 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

19:10:25 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 4.8 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
3.1 M     Trainable params
1.7 M     Non-trainable params
4.8 M     Total params


19:10:26 WARNING:true_pair_set has 82 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.5434083938598633,
 'manufacturer': 0.17624834179878235,
 'price': 0.2803432047367096}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.3961605584642233,
 'valid_f1_at_0.5': 0.7184466019417476,
 'valid_f1_at_0.7': 0.8,
 'valid_f1_at_0.9': 0.6984924623115578,
 'valid_pair_entity_ratio_at_0.3': 1.9956236323851204,
 'valid_pair_entity_ratio_at_0.5': 0.8402625820568927,
 'valid_pair_entity_ratio_at_0.7': 0.6367614879649891,
 'valid_pair_entity_ratio_at_0.9': 0.3588621444201313,
 'valid_precision_at_0.3': 0.24890350877192982,
 'valid_precision_at_0.5': 0.578125,
 'valid_precision_at_0.7': 0.7216494845360825,
 'valid_precision_at_0.9': 0.8475609756097561,
 'valid_recall_at_0.3': 0.9700854700854701,
 'valid_recall_at_0.5': 0.9487179487179487,
 'valid_recall_at_0.7': 0.8974358974358975,
 'valid_recall_at_0.9': 0.594017094017094}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.35901509134233517,
  'test_f1_at_0.5': 0.7058823529411764,
  'test_f1_at_0.7': 0.8007662835249041,
  'test_f1_at_0.9': 0.6666666666666667,
  'test_pair_entity_ratio_at_0.3': 2.2234273318872018,
  'test_pair_entity_ratio_at_0.5': 0.8568329718004338,
  'test_pair_entity_ratio_at_0.7': 0.6247288503253796,
  'test_pair_entity_ratio_at_0.9': 0.2993492407809111,
  'test_precision_at_0.3': 0.22048780487804878,
  'test_precision_at_0.5': 0.5620253164556962,
  'test_precision_at_0.7': 0.7256944444444444,
  'test_precision_at_0.9': 0.8985507246376812,
  'test_recall_at_0.3': 0.9658119658119658,
  'test_recall_at_0.5': 0.9487179487179487,
  'test_recall_at_0.7': 0.8931623931623932,
  'test_recall_at_0.9': 0.5299145299145299}]

## Testing manually 

In [22]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict, cluster_dict_to_filtered_row_dict

__, test_cluster_dict = id_pairs_to_cluster_mapping_and_dict(benchmark.test_true_pair_set)
test_row_dict = cluster_dict_to_filtered_row_dict(benchmark.row_dict, test_cluster_dict)
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set={id_ for id_, row in test_row_dict.items() if row['__source'] == 'left'},
    right_id_set={id_ for id_, row in test_row_dict.items() if row['__source'] == 'right'},
    batch_size=16
)

In [23]:
embedding_size = model.blocker_net.embedding_size

In [24]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [25]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 240 ms, sys: 0 ns, total: 240 ms
Wall time: 35.6 ms


In [26]:
%%time

sim_threshold = 0.7
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 270 ms, sys: 0 ns, total: 270 ms
Wall time: 31.7 ms


In [27]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

0.6247288503253796

In [30]:
from entity_embed.evaluation import precision_and_recall, f1_score

precision, recall = precision_and_recall(found_pair_set, benchmark.test_true_pair_set)
precision, recall

(0.7256944444444444, 0.8931623931623932)

In [31]:
f1_score(precision, recall)

0.8007662835249041

In [35]:
false_positives = list(found_pair_set - benchmark.test_true_pair_set)
len(false_positives)

79

In [36]:
false_negatives = list(benchmark.test_true_pair_set - found_pair_set)
len(false_negatives)

25

In [37]:
cos_similarity = lambda a, b: np.dot(a, b)

In [39]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            test_row_dict[id_left], test_row_dict[id_right]
        )
    )

(0.6632249,
 {'id': 17,
  'title': 'upg sgms 1000 incremental node',
  'manufacturer': 'sonic - systems - inc .',
  'price': '',
  '__source': 'left'},
 {'id': 3347,
  'title': 'sonicwall gms 1000n - incremental lic upg',
  'manufacturer': 'sonic - systems - inc .',
  'price': '62920 . 89',
  '__source': 'right'})

(0.59192604,
 {'id': 395,
  'title': 'world history ( win / mac ) ( jewel case )',
  'manufacturer': 'fogware publishing',
  'price': '9 . 99',
  '__source': 'left'},
 {'id': 4269,
  'title': 'high school world history ( pc / mac ) fogware',
  'manufacturer': '',
  'price': '9 . 99',
  '__source': 'right'})

(0.43140906,
 {'id': 207,
  'title': 'crystal reports 11 professional full product french',
  'manufacturer': 'business objects',
  'price': '909 . 66',
  '__source': 'left'},
 {'id': 2888,
  'title': 'crystal reports xi professional edition complete package',
  'manufacturer': '',
  'price': '537 . 57',
  '__source': 'right'})

(0.24009395,
 {'id': 1206,
  'title': 'contentbarrier x4 10 . 4 single user ( mac )',
  'manufacturer': 'intego',
  'price': '49 . 99',
  '__source': 'left'},
 {'id': 1900,
  'title': 'intego contentbarrier x4 10 . 4',
  'manufacturer': '',
  'price': '54 . 99',
  '__source': 'right'})

(0.41482356,
 {'id': 1360,
  'title': 'dragon naturally speaking standard v9',
  'manufacturer': 'nuance - communications - inc .',
  'price': '99 . 99',
  '__source': 'left'},
 {'id': 1973,
  'title': 'nuance communications inc . dragon ns standard v9',
  'manufacturer': 'nuance - communications - inc .',
  'price': '92 . 51',
  '__source': 'right'})

(0.54197437,
 {'id': 1190,
  'title': 'punch ! professional home design',
  'manufacturer': 'punch ! software',
  'price': '89 . 99',
  '__source': 'left'},
 {'id': 3789,
  'title': 'punch software 14100 punch ! home design suite professional architectural 1 user ( s ) pc',
  'manufacturer': '',
  'price': '59 . 97',
  '__source': 'right'})

(0.6443167,
 {'id': 161,
  'title': 'gods : lands of infinity special edition',
  'manufacturer': 'strategy first',
  'price': '29 . 99',
  '__source': 'left'},
 {'id': 2854,
  'title': 'gods : lands of infinity spec',
  'manufacturer': '',
  'price': '16 . 99',
  '__source': 'right'})

(0.45422035,
 {'id': 273,
  'title': 'quickbooks credit card processing kit 3 . 0',
  'manufacturer': 'intuit',
  'price': '39 . 95',
  '__source': 'left'},
 {'id': 1371,
  'title': 'qb credit card kit 3 . 0',
  'manufacturer': 'intuit',
  'price': '25 . 99',
  '__source': 'right'})

(0.23871167,
 {'id': 726,
  'title': 'simply put software data eliminator',
  'manufacturer': 'simply - put - software',
  'price': '',
  '__source': 'left'},
 {'id': 4151,
  'title': 'simply put software llc de905 - s data elminator ( win 95 98 me nt 2000 xp )',
  'manufacturer': 'simply - put - software',
  'price': '34 . 97',
  '__source': 'right'})

(0.1845403,
 {'id': 812,
  'title': 'extensis smartscale 1 - user pxe - 11433 )',
  'manufacturer': 'extensis corporation',
  'price': '',
  '__source': 'left'},
 {'id': 4160,
  'title': 'onone software pxe - 11433 pxl smartscale elect 1 - user english mac / win',
  'manufacturer': 'onone software',
  'price': '158 . 99',
  '__source': 'right'})